In [ ]:
!kaggle datasets download moazeldsokyx/bbc-news

Dataset URL: https://www.kaggle.com/datasets/moazeldsokyx/bbc-news
License(s): unknown
bbc-news.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip -o bbc-news.zip

Archive:  bbc-news.zip
  inflating: bbc-text.csv            


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df=pd.read_csv('bbc-text.csv')

In [ ]:
df.shape

(2225, 2)

In [ ]:
df.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [ ]:
df.category.value_counts()

category
sport            511
business         510
politics         417
tech             401
entertainment    386
Name: count, dtype: int64

In [ ]:
df.category.unique()

array(['tech', 'business', 'sport', 'entertainment', 'politics'],
      dtype=object)

In [ ]:
labels=['tech', 'business', 'sport', 'entertainment', 'politics']
labels

['tech', 'business', 'sport', 'entertainment', 'politics']

In [ ]:
df.isnull().sum()

category    0
text        0
dtype: int64

In [ ]:
df.duplicated().sum()

99

In [ ]:
df.head()

,category,text,vector
0,0,tv future in the hands of viewers with home th...,"[0.0031586075, 0.010951136, 0.0011938965, 0.03..."
1,1,worldcom boss left books alone former worldcom...,"[-0.010469076, 0.0073564043, -0.0048506577, 0...."
2,2,tigers wary of farrell gamble leicester say th...,"[0.00036933756, 0.014444627, 0.0017897796, 0.0..."
3,2,yeading face newcastle in fa cup premiership s...,"[-0.008516042, 0.008820114, 0.026090756, 0.022..."
4,4,ocean s twelve raids box office ocean s twelve...,"[-0.006611159, 0.014178862, -0.0066637546, 0.0..."
5,3,howard hits back at mongrel jibe michael howar...,"[-0.0066210716, 0.015717264, 0.006659347, 0.04..."
6,3,blair prepares to name poll date tony blair is...,"[-0.012308735, 0.014406368, -0.0040963544, 0.0..."
7,2,henman hopes ended in dubai third seed tim hen...,"[-0.004276141, 0.017335914, -0.0033996275, 0.0..."
8,2,wilkinson fit to face edinburgh england captai...,"[-0.010723096, 0.019835413, 0.0032756838, 0.02..."
9,4,last star wars not for children the sixth and ...,"[0.01600349, 0.01154447, 0.010081989, 0.038551..."


In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [ ]:
df.duplicated().sum()

0

In [ ]:
df.category=df.category.map({'tech':0,'business':1,'sport':2,'politics':3,'entertainment':4})

In [ ]:
import gensim.downloader as api
w2v=api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [ ]:
def lower(text):
    return text.lower()
df.text=df.text.apply(lower)

In [ ]:
import re
def remove_url(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)
df.text=df.text.apply(remove_url)

In [ ]:
import nltk
import string
import re

In [ ]:
# import the inflect library
import inflect
p = inflect.engine()

# convert number into words
def convert_number(text):
    # split string into list of words
    temp_str = text.split()
    # initialise empty list
    new_string = []

    for word in temp_str:
        # if word is a digit, convert the digit
        # to numbers and append into the new_string list
        if word.isdigit():
            temp = p.number_to_words(word)
            new_string.append(temp)

        # append the word as it is
        else:
            new_string.append(word)

    # join the words of new_string to form a string
    temp_str = ' '.join(new_string)
    return temp_str

df.text=df.text.apply(convert_number)

In [ ]:
# remove whitespace from text
def remove_whitespace(text):
    return  " ".join(text.split())
df.text=df.text.apply(remove_whitespace)

In [ ]:
def remove_punctuation(text):
  pattern = r'[.,!$()@*%#\n]'
  return re.sub(pattern, '', text)

df.text=df.text.apply(remove_punctuation)




In [ ]:
def preandvec(text):
  doc=nlp(text)
  filtered=[]
  for word in doc:
    if word.is_stop or word.is_punct:
      continue
    filtered.append(word.lemma_)
  return w2v.get_mean_vector(filtered)

df['vector']=df.text.apply(preandvec)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df.vector,df.category,test_size=0.2,random_state=2022)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((1700,), (426,), (1700,), (426,))

In [ ]:
x_train2d=np.stack(x_train)
x_test2d=np.stack(x_test)
x_train2d.shape,x_test2d.shape

((1700, 300), (426, 300))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model=DecisionTreeClassifier()
model.fit(x_train2d,y_train)
y_pred=model.predict(x_test2d)
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(model.score(x_train2d,y_train))
print(model.score(x_test2d,y_test))


              precision    recall  f1-score   support

           0       0.74      0.73      0.73        51
           1       0.83      0.87      0.85       112
           2       0.89      0.85      0.87       108
           3       0.70      0.75      0.73        77
           4       0.82      0.77      0.79        78

    accuracy                           0.81       426
   macro avg       0.80      0.79      0.79       426
weighted avg       0.81      0.81      0.81       426

[[37  2  4  4  4]
 [ 4 97  1  8  2]
 [ 5  3 92  4  4]
 [ 1 12  3 58  3]
 [ 3  3  3  9 60]]
1.0
0.8075117370892019


In [ ]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier()
model.fit(x_train2d,y_train)
y_pred=model.predict(x_test2d)
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(model.score(x_train2d,y_train))
print(model.score(x_test2d,y_test))


              precision    recall  f1-score   support

           0       0.94      0.90      0.92        51
           1       0.94      0.97      0.96       112
           2       0.97      0.97      0.97       108
           3       0.92      0.91      0.92        77
           4       0.95      0.94      0.94        78

    accuracy                           0.95       426
   macro avg       0.94      0.94      0.94       426
weighted avg       0.95      0.95      0.95       426

[[ 46   1   1   1   2]
 [  0 109   0   3   0]
 [  0   1 105   0   2]
 [  1   5   1  70   0]
 [  2   0   1   2  73]]
1.0
0.9460093896713615


In [ ]:
import pickle
with open('RFnews.pkl','wb') as f:
  pickle.dump(model,f)

In [ ]:
from xgboost import XGBClassifier
model=XGBClassifier()
model.fit(x_train2d,y_train)
y_pred=model.predict(x_test2d)
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(model.score(x_train2d,y_train))
print(model.score(x_test2d,y_test))


              precision    recall  f1-score   support

           0       0.96      0.94      0.95        51
           1       0.97      0.96      0.97       112
           2       0.98      0.98      0.98       108
           3       0.92      0.94      0.93        77
           4       0.92      0.94      0.93        78

    accuracy                           0.96       426
   macro avg       0.95      0.95      0.95       426
weighted avg       0.96      0.96      0.96       426

[[ 48   1   0   0   2]
 [  0 108   0   4   0]
 [  0   0 106   0   2]
 [  0   2   1  72   2]
 [  2   0   1   2  73]]
1.0
0.9553990610328639


In [ ]:
import pickle
with open('XGnews.pkl','wb') as f:
  pickle.dump(model,f)

In [ ]:
from sklearn.svm import SVC
model=SVC(kernel='rbf')
model.fit(x_train2d,y_train)
y_pred=model.predict(x_test2d)
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(model.score(x_train2d,y_train))
print(model.score(x_test2d,y_test))

              precision    recall  f1-score   support

           0       0.93      0.98      0.95        51
           1       0.98      0.96      0.97       112
           2       0.99      1.00      1.00       108
           3       0.94      0.96      0.95        77
           4       1.00      0.95      0.97        78

    accuracy                           0.97       426
   macro avg       0.97      0.97      0.97       426
weighted avg       0.97      0.97      0.97       426

[[ 50   1   0   0   0]
 [  1 108   0   3   0]
 [  0   0 108   0   0]
 [  1   1   1  74   0]
 [  2   0   0   2  74]]
0.9835294117647059
0.971830985915493


In [ ]:
import pickle
with open('SVCnews.pkl','wb') as f:
  pickle.dump(model,f)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model=KNeighborsClassifier(n_neighbors=5)
model.fit(x_train2d,y_train)
y_pred=model.predict(x_test2d)
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(model.score(x_train2d,y_train))
print(model.score(x_test2d,y_test))


              precision    recall  f1-score   support

           0       0.91      0.94      0.92        51
           1       0.96      0.96      0.96       112
           2       0.97      1.00      0.99       108
           3       0.91      0.94      0.92        77
           4       0.99      0.90      0.94        78

    accuracy                           0.95       426
   macro avg       0.95      0.95      0.95       426
weighted avg       0.95      0.95      0.95       426

[[ 48   1   1   0   1]
 [  2 107   1   2   0]
 [  0   0 108   0   0]
 [  1   3   1  72   0]
 [  2   1   0   5  70]]
0.9670588235294117
0.9507042253521126


In [ ]:
import pickle
with open('KNNnews.pkl','wb') as f:
  pickle.dump(model,f)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
model= GradientBoostingClassifier()
model.fit(x_train2d,y_train)
y_pred=model.predict(x_test2d)
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(model.score(x_train2d,y_train))
print(model.score(x_test2d,y_test))


              precision    recall  f1-score   support

           0       0.98      0.92      0.95        51
           1       0.97      0.98      0.98       112
           2       0.98      0.98      0.98       108
           3       0.95      0.95      0.95        77
           4       0.94      0.96      0.95        78

    accuracy                           0.96       426
   macro avg       0.96      0.96      0.96       426
weighted avg       0.96      0.96      0.96       426

[[ 47   1   1   1   1]
 [  0 110   0   2   0]
 [  0   0 106   0   2]
 [  0   1   1  73   2]
 [  1   1   0   1  75]]
1.0
0.9647887323943662


In [ ]:
import pickle
with open('GRBOOSINGnews.pkl','wb') as f:
  pickle.dump(model,f)

In [ ]:
newmodel=pickle.load(open('RFnews.pkl','rb'))

In [ ]:
text="Share market crashes"
text=lower(text)
text=remove_url(text)
text=convert_number(text)
text=remove_whitespace(text)
text=remove_punctuation(text)
p=preandvec(text)
ans=newmodel.predict(p.reshape(1,-1))[0]
print(labels[ans])

business


In [ ]:
text="indian wins world cup"
text=lower(text)
text=remove_url(text)
text=convert_number(text)
text=remove_whitespace(text)
text=remove_punctuation(text)
p=preandvec(text)
ans=newmodel.predict(p.reshape(1,-1))[0]
print(labels[ans])

sport


In [ ]:
text="iron man"
text=lower(text)
text=remove_url(text)
text=convert_number(text)
text=remove_whitespace(text)
text=remove_punctuation(text)
p=preandvec(text)
ans=newmodel.predict(p.reshape(1,-1))[0]
print(labels[ans])

entertainment


In [ ]:
text="tv future in the hands of viewers"
text=lower(text)
text=remove_url(text)
text=convert_number(text)
text=remove_whitespace(text)
text=remove_punctuation(text)
p=preandvec(text)
ans=newmodel.predict(p.reshape(1,-1))[0]
print(labels[ans])

tech


In [ ]:
text="Shahrukh khan"
text=lower(text)
text=remove_url(text)
text=convert_number(text)
text=remove_whitespace(text)
text=remove_punctuation(text)
p=preandvec(text)
ans=newmodel.predict(p.reshape(1,-1))[0]
print(labels[ans])

politics
